<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

np.random.seed(101)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
farsi_path = "drive/MyDrive/final_final.csv"
english_path = "drive/MyDrive/my_final_english_hate.csv"
eng = pd.read_csv(english_path)
farsi = pd.read_csv(farsi_path)

In [ ]:
farsi = farsi[['google', 'mbart', 'ref3']]

In [ ]:
def deEmojify(row):
    text = row.tweet
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
eng = eng.drop("Unnamed: 0", axis=1)
eng['tweet'] = eng.apply(deEmojify, axis=1)
eng = eng.sample(frac=1).reset_index(drop=True)
eng.head()

,tweet,class
0,RT @USER You a hoe if you give your number to ...,1
1,Best rider? That goes to a blaxican.,0
2,@USER hes just a friggin idiot that can say an...,1
3,That niggas retarded,1
4,Why is my mother bothering me about the trash?...,0


In [ ]:
def deEmojifyGoogle(row):
    text = row.google
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
def deEmojifyMbart(row):
    text = row.mbart
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
farsi = farsi.dropna()
farsi['google'] = farsi.apply(deEmojifyGoogle, axis=1)
farsi['mbart'] = farsi.apply(deEmojifyMbart, axis=1)
farsi = farsi.sample(frac=1).reset_index(drop=True)
farsi.head()

,google,mbart,ref3
0,Bvdymyr and tell us that we're ten more comfor...,"We were like, we're going to make it easier fo...",0.0
1,We happened upon the streets Gzart.\nA city\nW...,We sat in that alley that had fallen over — it...,0.0
2,The number of male and female have a very diff...,The number of boys and girls doesn't seem that...,0.0
3,If Arab students are female doctor Persian He ...,"If you're an Arab student, the Ph.D. student o...",0.0
4,I was jumping up Qtlm a hedgehog.\nThe prayer ...,It wasn't long before I was going to be the ki...,0.0


In [ ]:
x_train = eng[['tweet']]
y_train = eng[['class']]

In [ ]:
x_google = farsi[['google']]
y_google = farsi[['ref3']]
x_mbart = farsi[['mbart']]
y_mbart = farsi[['ref3']]

In [ ]:
y_google.columns = ['class']
y_mbart.columns = ['class'] 

In [ ]:
x_train['tweet']    = [entry.lower() for entry in x_train['tweet']]
x_google['google'] = [entry.lower() for entry in x_google['google']]
x_mbart['mbart']   = [entry.lower() for entry in x_mbart['mbart']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
x_train['tweet']    = [word_tokenize(entry) for entry in x_train['tweet']]
x_google['google'] = [word_tokenize(entry) for entry in x_google['google']]
x_mbart['mbart']   = [word_tokenize(entry) for entry in x_mbart['mbart']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [ ]:
def clean(pd, name):
  for index,entry in enumerate(pd[name]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    pd.loc[index,'text_final'] = str(Final_words)
  return pd

In [ ]:
x_train = clean(x_train, 'tweet')
x_google = clean(x_google, 'google')
x_mbart = clean(x_mbart, 'mbart')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

In [ ]:
tune_X_google, Test_X_google, tune_Y_google, Test_Y_google = model_selection.train_test_split(x_google['text_final'],y_google ,test_size=0.4, random_state=101)

tune_X_mbart, Test_X_mbart, tune_Y_mbart, Test_Y_mbart  = model_selection.train_test_split(x_mbart['text_final'],y_mbart ,test_size=0.4, random_state=101)


In [ ]:
tune_Y_google = tune_Y_google.astype(dtype="int")  
Test_Y_google = Test_Y_google.astype(dtype="int")
tune_Y_mbart = tune_Y_mbart.astype(dtype="int")
Test_Y_mbart = Test_Y_mbart.astype(dtype="int")

In [ ]:
tune_X_google

3214    ['guy', 'monster', 'time', 'pass', 'demon', 'p...
1262    ['late', 'late', 'cable', 'direct', 'czech', '...
2069    ['khbrayn', 'week', 'hell', 'late', 'hdadksyf'...
322     ['know', 'manvk', 'khodabakhshian', 'go', 'mea...
4294    ['reed', 'ya', 'club', 'sorry', 'sorry', 'say'...
                              ...                        
4171    ['ready', 'help', 'account', 'number', 'announ...
599           ['simple', 'nemishe', 'little', 'shrewish']
1361    ['pretty', 'cool', 'one', 'hour', 'delete', 'k...
1547    ['nothing', 'never', 'wan', 'na', 'n', 'pituit...
4959                                    ['think', 'like']
Name: text_final, Length: 3120, dtype: object

In [ ]:
x_train = x_train['text_final']
x_train

0        ['rt', 'user', 'hoe', 'give', 'number', 'anoth...
1                      ['best', 'rider', 'go', 'blaxican']
2        ['user', 'he', 'friggin', 'idiot', 'say', 'any...
3                                      ['nigga', 'retard']
4        ['mother', 'bother', 'trash', 'day', 'leave', ...
                               ...                        
10848                                   ['user', 'faggot']
10849    ['rangzen', 'people', 'stop', 'fake', 'activis...
10850    ['user', 'orange', 'menace', 'user', 'side', '...
10851                           ['like', 'bitch', 'nigga']
10852    ['user', 'lmao', 'yes', 'kind', 'like', 'ghett...
Name: text_final, Length: 10853, dtype: object

In [ ]:
max_features = 500

In [ ]:
Tfidf_vect_x_train = TfidfVectorizer(max_features=max_features)
Tfidf_vect_x_train.fit(x_train)

Train_X_Tfidf      = Tfidf_vect_x_train.transform(x_train)
tune_google_Tfidf  = Tfidf_vect_x_train.transform(tune_X_google)
tune_mbart_Tfidf   = Tfidf_vect_x_train.transform(tune_X_mbart)
test_google_Tfidf  = Tfidf_vect_x_train.transform(Test_X_google)
Test_mbart_Tfidf   = Tfidf_vect_x_train.transform(Test_X_mbart)

In [ ]:
SVM = svm.SVC(C=0.5, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predictions_NB = SVM.predict(tune_google_Tfidf)
print(classification_report(tune_Y_google , predictions_NB))

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2644
           1       0.54      0.16      0.24       476

    accuracy                           0.85      3120
   macro avg       0.70      0.57      0.58      3120
weighted avg       0.82      0.85      0.81      3120



In [ ]:
# x_train = x_train.append(tune_X_google)
# y_train = y_train.append(tune_Y_google)
# x_train.reset_index(inplace=True, drop=True)
# y_train.reset_index(inplace=True, drop=True)
# x_train
x_train = x_train.append(tune_X_mbart)
y_train = y_train.append(tune_Y_mbart)
x_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
x_train

0        ['rt', 'user', 'hoe', 'give', 'number', 'anoth...
1                      ['best', 'rider', 'go', 'blaxican']
2        ['user', 'he', 'friggin', 'idiot', 'say', 'any...
3                                      ['nigga', 'retard']
4        ['mother', 'bother', 'trash', 'day', 'leave', ...
                               ...                        
13968    ['go', 'help', 'get', 'account', 'number', 'sa...
13969                                           ['simple']
13970    ['really', 'fun', 'spend', 'hour', 'diving', '...
13971                               ['say', 'pray', 'god']
13972                                    ['think', 'coma']
Name: text_final, Length: 13973, dtype: object

In [ ]:
x_train

0        ['rt', 'user', 'hoe', 'give', 'number', 'anoth...
1                      ['best', 'rider', 'go', 'blaxican']
2        ['user', 'he', 'friggin', 'idiot', 'say', 'any...
3                                      ['nigga', 'retard']
4        ['mother', 'bother', 'trash', 'day', 'leave', ...
                               ...                        
13968    ['go', 'help', 'get', 'account', 'number', 'sa...
13969                                           ['simple']
13970    ['really', 'fun', 'spend', 'hour', 'diving', '...
13971                               ['say', 'pray', 'god']
13972                                    ['think', 'coma']
Name: text_final, Length: 13973, dtype: object

In [ ]:
y_train

,class
0,1
1,0
2,1
3,1
4,0
...,...
13968,1
13969,1
13970,0
13971,0


In [ ]:
# Tfidf_vect_x_train = TfidfVectorizer(max_features=max_features)
# Tfidf_vect_x_train.fit(x_train)
# Train_X_Tfidf      = Tfidf_vect_x_train.transform(x_train)


Tfidf_vect_x_train = TfidfVectorizer(max_features=max_features)
Tfidf_vect_x_train.fit(x_train)
Train_X_Tfidf      = Tfidf_vect_x_train.transform(x_train)

In [ ]:
# test_google_Tfidf  = Tfidf_vect_x_train.transform(Test_X_google)

test_mbart_Tfidf  = Tfidf_vect_x_train.transform(Test_X_mbart)

In [ ]:
# SVM = svm.SVC(C=0.5, kernel='linear', degree=3, gamma='auto')
# SVM.fit(Train_X_Tfidf,y_train)


SVM = svm.SVC(C=0.5, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
# predictions_svm = SVM.predict(test_google_Tfidf)

# # # result of test for google translate
# print("result of test for google translate")
# print(classification_report( Test_Y_google, predictions_svm))



predictions_svm = SVM.predict(test_mbart_Tfidf)

# # result of test for mbart translate
print("result of test for mbart translate")
print(classification_report( Test_Y_mbart, predictions_svm))

result of test for mbart translate
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1792
           1       0.31      0.01      0.03       288

    accuracy                           0.86      2080
   macro avg       0.59      0.50      0.48      2080
weighted avg       0.79      0.86      0.80      2080



**not use english dataset**


In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=max_features)
Tfidf_vect.fit(tune_X_google)
Train_X_Tfidf = Tfidf_vect.transform(tune_X_google)
Test_X_Tfidf = Tfidf_vect.transform(Test_X_google)

In [ ]:
SVM = svm.SVC(C=0.5, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,tune_Y_google)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("test if we dont use english train dataset")
print(classification_report( Test_Y_google, predictions_SVM))

test if we dont use english train dataset
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1792
           1       0.70      0.06      0.10       288

    accuracy                           0.87      2080
   macro avg       0.78      0.53      0.52      2080
weighted avg       0.84      0.87      0.81      2080



In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=max_features)
Tfidf_vect.fit(tune_X_mbart)
Train_X_Tfidf = Tfidf_vect.transform(tune_X_mbart)
Test_X_Tfidf = Tfidf_vect.transform(Test_X_mbart)

In [ ]:
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("test if we dont use english train dataset")
print(classification_report( Test_Y_google, predictions_SVM))

test if we dont use english train dataset
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      1792
           1       0.12      0.01      0.02       288

    accuracy                           0.85      2080
   macro avg       0.49      0.50      0.47      2080
weighted avg       0.76      0.85      0.79      2080

